In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'FDITTVSJQZVB3D0CHDYHK1INFTXOR0UYZ51XAKUDW1IGQLV5' # your Foursquare ID
CLIENT_SECRET = '2JT5IL5I0DGROK14M3PWMBG54MRW5QSKTO4DJCOKJRYAWKJ3' # your Foursquare Secret
ACCESS_TOKEN = '05B50VTMVA2NKP0V52WT5LL5IEKEQVXQVLA1BZSRMWSH0UOP' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FDITTVSJQZVB3D0CHDYHK1INFTXOR0UYZ51XAKUDW1IGQLV5
CLIENT_SECRET:2JT5IL5I0DGROK14M3PWMBG54MRW5QSKTO4DJCOKJRYAWKJ3


In [10]:
address = '5900 Centreville Crest Ln, Centreville, VA 20121'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.838088850000005 -77.43596011231011


In [11]:
search_query = 'coffee'
radius = 500
print(search_query + ' .... OK!')

coffee .... OK!


In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FDITTVSJQZVB3D0CHDYHK1INFTXOR0UYZ51XAKUDW1IGQLV5&client_secret=2JT5IL5I0DGROK14M3PWMBG54MRW5QSKTO4DJCOKJRYAWKJ3&ll=38.838088850000005,-77.43596011231011&oauth_token=05B50VTMVA2NKP0V52WT5LL5IEKEQVXQVLA1BZSRMWSH0UOP&v=20180604&query=coffee&radius=500&limit=30'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6115244438aa41382fa5ef23'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '5f405915ea03aa5dbdfa8df9',
    'name': 'Coffee Project',
    'location': {'address': '5825 Trinity Pkwy',
     'lat': 38.839608,
     'lng': -77.438014,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.839608,
       'lng': -77.438014}],
     'distance': 245,
     'postalCode': '20120',
     'cc': 'US',
     'city': 'Centreville',
     'state': 'VA',
     'country': 'United States',
     'formattedAddress': ['5825 Trinity Pkwy', 'Centreville, VA 20120']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1628775492',
    'hasPerk': False},
   {'i

In [14]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-14-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,5f405915ea03aa5dbdfa8df9,Coffee Project,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1628775492,False,5825 Trinity Pkwy,38.839608,-77.438014,"[{'label': 'display', 'lat': 38.839608, 'lng':...",245,20120,US,Centreville,VA,United States,"[5825 Trinity Pkwy, Centreville, VA 20120]"
1,4b409e95f964a5205ab925e3,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1628775492,False,14155 Saint Germain Dr,38.836339,-77.437358,"[{'label': 'display', 'lat': 38.83633861, 'lng...",229,20121,US,Centreville,VA,United States,"[14155 Saint Germain Dr, Centreville, VA 20121]"


In [15]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Coffee Project,Coffee Shop,5825 Trinity Pkwy,38.839608,-77.438014,"[{'label': 'display', 'lat': 38.839608, 'lng':...",245,20120,US,Centreville,VA,United States,"[5825 Trinity Pkwy, Centreville, VA 20120]",5f405915ea03aa5dbdfa8df9
1,Starbucks,Coffee Shop,14155 Saint Germain Dr,38.836339,-77.437358,"[{'label': 'display', 'lat': 38.83633861, 'lng...",229,20121,US,Centreville,VA,United States,"[14155 Saint Germain Dr, Centreville, VA 20121]",4b409e95f964a5205ab925e3


In [16]:
dataframe_filtered.name

0    Coffee Project
1         Starbucks
Name: name, dtype: object

In [17]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


In [19]:
# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Centreville',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

In [20]:
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


In [30]:
venues_map

In [24]:
search_query = 'gym'
radius = 500
print(search_query + ' .... OK!')

gym .... OK!


In [25]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FDITTVSJQZVB3D0CHDYHK1INFTXOR0UYZ51XAKUDW1IGQLV5&client_secret=2JT5IL5I0DGROK14M3PWMBG54MRW5QSKTO4DJCOKJRYAWKJ3&ll=38.838088850000005,-77.43596011231011&oauth_token=05B50VTMVA2NKP0V52WT5LL5IEKEQVXQVLA1BZSRMWSH0UOP&v=20180604&query=gym&radius=500&limit=30'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '611524fe212946064a02b8c9'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4eefd3aaf9ab3edffb200450',
    'name': '[gYm TiMe}',
    'location': {'address': 'Post Corner',
     'lat': 38.84156129526325,
     'lng': -77.43395553471379,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.84156129526325,
       'lng': -77.43395553471379}],
     'distance': 423,
     'postalCode': '20120',
     'cc': 'US',
     'city': 'Centreville',
     'state': 'VA',
     'country': 'United States',
     'formattedAddress': ['Post Corner', 'Centreville, VA 20120']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1628775678',
    'hasPerk': False},
   {'id': '4d5ace

In [27]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-27-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4eefd3aaf9ab3edffb200450,[gYm TiMe},"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1628775678,False,Post Corner,38.841561,-77.433956,"[{'label': 'display', 'lat': 38.84156129526325...",423,20120,US,Centreville,VA,United States,"[Post Corner, Centreville, VA 20120]",NaN
1,4d5ace991d4a8cfa53c09ced,Woodway Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1628775678,False,5751 Wood Meadow Way,38.842589,-77.435364,"[{'label': 'display', 'lat': 38.84258947768470...",503,20120,US,Centreville,VA,United States,"[5751 Wood Meadow Way (Trinity Parkway), Centr...",Trinity Parkway


In [28]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,[gYm TiMe},Gym,Post Corner,38.841561,-77.433956,"[{'label': 'display', 'lat': 38.84156129526325...",423,20120,US,Centreville,VA,United States,"[Post Corner, Centreville, VA 20120]",NaN,4eefd3aaf9ab3edffb200450
1,Woodway Gym,Gym,5751 Wood Meadow Way,38.842589,-77.435364,"[{'label': 'display', 'lat': 38.84258947768470...",503,20120,US,Centreville,VA,United States,"[5751 Wood Meadow Way (Trinity Parkway), Centr...",Trinity Parkway,4d5ace991d4a8cfa53c09ced


In [29]:
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)
